In [1]:
!pip install -q openai gradio

import json
import gradio as gr
from openai import OpenAI

# 🔐 PASTE YOUR OPENAI KEY HERE
OPENAI_API_KEY = "PASTE_YOUR_OPENAI_KEY_HERE"

if not OPENAI_API_KEY or OPENAI_API_KEY == "PASTE_YOUR_OPENAI_KEY_HERE":
    raise ValueError("Please paste your OpenAI API key.")

client = OpenAI(api_key=OPENAI_API_KEY)

def build_prompt(profile):
    lang = profile.get("language", "English")

    if lang == "العربية":
        language_instruction = (
            "اكتب جميع النصوص باللغة العربية الفصحى (أيام الأسبوع، أسماء التمارين، "
            "ملاحظات التمرين، وصف الوجبات، والتحذيرات). يجب أن تبقى أسماء المفاتيح في JSON "
            "باللغة الإنجليزية تمامًا كما هي (weekly_workout, weekly_meals, disclaimers, day, "
            "focus, exercises, name, sets, reps, notes, meal, description, approx_calories)."
        )
    else:
        language_instruction = (
            "Write all text (day names, exercise names, notes, meal descriptions, disclaimers) "
            "in clear English. Keep the JSON keys exactly as shown (weekly_workout, weekly_meals, "
            "disclaimers, day, focus, exercises, name, sets, reps, notes, meal, description, "
            "approx_calories)."
        )

    return f"""
You are a certified fitness trainer and nutrition coach.

Return ONLY valid JSON. No explanations. No text before or after.
Return ONLY a raw JSON object. No markdown, no code blocks, no comments, no explanation text.


{language_instruction}

Rules:
- The "weekly_workout" array MUST contain EXACTLY {profile['training_days']} workout days.
- Each workout day MUST follow this structure:
  {{
    "day": "Monday",
    "focus": "Main muscle group",
    "exercises": [
      {{"name": "Exercise", "sets": 3, "reps": "10-12"}}
    ],
    "notes": "Short tip"
  }}

- The "weekly_meals" array can contain any reasonable number of items.
- The output MUST be valid JSON.

JSON FORMAT (example only):

{{
  "weekly_workout": [
    {{
      "day": "Monday",
      "focus": "Full body",
      "exercises": [
        {{"name": "Squats", "sets": 3, "reps": "10-12"}}
      ],
      "notes": "Warm up properly"
    }}
  ],
  "weekly_meals": [
    {{
      "day": "Monday",
      "meal": "Breakfast",
      "description": "Meal description",
      "approx_calories": 350
    }}
  ],
  "disclaimers": [
    "Not medical advice",
    "Consult a professional"
  ]
}}

User Profile:
- Language: {lang}
- Gender: {profile['gender']}
- Age: {profile['age']}
- Height: {profile['height_cm']} cm
- Weight: {profile['weight_kg']} kg
- Goal: {profile['goal']}
- Training Days: {profile['training_days']}
- Experience: {profile['experience']}
- Diet: {profile.get('diet_preference', '')}
"""




import re
import json



def generate_plan(profile):
    try:
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=build_prompt(profile),
            max_output_tokens=1400,
        )

        raw_text = response.output_text.strip()

        # ✅ Extract first valid JSON block
        json_start = raw_text.find("{")
        json_end = raw_text.rfind("}") + 1
        clean_json = raw_text[json_start:json_end]

        try:
            return json.loads(clean_json)

        except json.JSONDecodeError:
            # ✅ AUTO JSON REPAIR (removes common LLM mistakes)
            repaired = (
                clean_json
                .replace("،", ",")
                .replace(",]", "]")
                .replace(",}", "}")
            )

            return json.loads(repaired)

    except Exception as e:
        return {"error": str(e)}






def format_output(plan, language="English"):
    # Headings depending on language
    if language == "العربية":
        title_workout = "🏋️ خطة التمرين الأسبوعية"
        title_meals = "🍽️ خطة الوجبات الأسبوعية"
        title_disclaimers = "⚠️ تنبيهات"
        day_bullet = "📅"
    else:
        title_workout = "🏋️ WEEKLY WORKOUT PLAN"
        title_meals = "🍽️ WEEKLY MEAL PLAN"
        title_disclaimers = "⚠️ DISCLAIMERS"
        day_bullet = "📅"

    # ✅ BMI block
    if language == "العربية":
        bmi_text = (
            f"📏 مؤشر كتلة الجسم (BMI): {plan.get('bmi', '')}\n"
            f"🎯 الوزن المثالي: من {plan.get('ideal_min', '')} إلى {plan.get('ideal_max', '')} كجم\n\n"
        )
    else:
        bmi_text = (
            f"📏 BMI: {plan.get('bmi', '')}\n"
            f"🎯 Ideal weight range: {plan.get('ideal_min', '')} – {plan.get('ideal_max', '')} kg\n\n"
        )

    text = bmi_text + f"{title_workout}\n\n"



    for day in plan["weekly_workout"]:
        text += f"{day_bullet} {day['day']} – {day['focus']}\n"
        for ex in day["exercises"]:
            text += f"• {ex['name']} — {ex['sets']} × {ex['reps']}\n"
        text += f"📝 {day.get('notes', '')}\n\n"

    text += f"\n{title_meals}\n\n"

    for meal in plan["weekly_meals"]:
        text += f"{day_bullet} {meal['day']} – {meal['meal']}\n"
        text += f"🥗 {meal['description']} (~{meal['approx_calories']} kcal)\n\n"

    text += f"\n{title_disclaimers}\n"
    for d in plan["disclaimers"]:
        text += f"- {d}\n"

    return text



LANG = {
    "English": {
        "gender": "Gender",
        "gender_choices": ["Female", "Male"],

        "goal": "Goal",
        "goal_choices": ["Lose fat", "Maintain", "Gain muscle"],

        "experience": "Experience Level",
        "experience_choices": ["Beginner", "Intermediate"],

        "diet": "Diet Preference",
        "diet_choices": ["No restriction", "High protein", "Vegetarian"],

        "age": "Age",
        "height": "Height (cm)",
        "weight": "Weight (kg)",
        "days": "Training Days Per Week",

        "submit": "Submit",
        "clear": "Clear",

        "title": "FitPlanner – AI Workout & Meal Planner",
        "desc": "A smart AI-powered fitness & nutrition planner for your graduation project.",
        "output": "Your Personalized Weekly Plan",
    },

    "العربية": {
        "gender": "الجنس",
        "gender_choices": ["أنثى", "ذكر"],

        "goal": "الهدف",
        "goal_choices": ["إنقاص الدهون", "المحافظة", "زيادة العضلات"],

        "experience": "مستوى الخبرة",
        "experience_choices": ["مبتدئ", "متوسط"],

        "diet": "النظام الغذائي",
        "diet_choices": ["بدون قيود", "عالي البروتين", "نباتي"],

        "age": "العمر",
        "height": "الطول (سم)",
        "weight": "الوزن (كجم)",
        "days": "عدد أيام التمرين بالأسبوع",

        "submit": "إنشاء الخطة",
        "clear": "مسح",

        "title": "FitPlanner – مخطط التمارين والوجبات بالذكاء الاصطناعي",
        "desc": "مخطط ذكي للتمارين والتغذية لمشروع التخرج.",
        "output": "خطة الأسبوع الشخصية",
    }
}




def calculate_bmi(height_cm, weight_kg):
    height_m = height_cm / 100
    bmi = weight_kg / (height_m ** 2)

    # Ideal weight range (BMI 18.5–24.9)
    min_ideal = 18.5 * (height_m ** 2)
    max_ideal = 24.9 * (height_m ** 2)

    return round(bmi, 2), round(min_ideal, 1), round(max_ideal, 1)






def app_ui(language, gender, age, height, weight, goal, days, experience, diet):

    # ✅ Validation
    if age is None or age == "" or int(age) <= 0:
        return "❌ Please enter a valid AGE."

    if height is None or height == "" or int(height) <= 0:
        return "❌ Please enter a valid HEIGHT."

    if weight is None or weight == "" or int(weight) <= 0:
        return "❌ Please enter a valid WEIGHT."

    training_days = int(round(float(days)))

    # ✅ ✅ ✅ THIS IS THE EXACT BMI INSERTION POINT ✅ ✅ ✅
    bmi, min_w, max_w = calculate_bmi(int(height), int(weight))

    profile = {
        "language": language,
        "gender": gender,
        "age": int(age),
        "height_cm": int(height),
        "weight_kg": int(weight),
        "goal": goal,
        "training_days": training_days,
        "experience": experience,
        "diet_preference": diet,


    }

    plan = generate_plan(profile)

    if "error" in plan:
        return f"❌ ERROR:\n{plan['error']}"
      # ✅ attach BMI info to the plan dict used by format_output
    plan["bmi"] = bmi
    plan["ideal_min"] = min_w
    plan["ideal_max"] = max_w

    return format_output(plan, language)






css = """
body {
  background: linear-gradient(135deg, #0f172a, #020617);
}

textarea, input {
  font-size: 16px !important;
}

/* ✅ RTL for the entire UI (form + texts) */
.rtl {
  direction: rtl;
  text-align: right;
}

/* ✅ Center ONLY the output result */
.center-output textarea {
  text-align: center !important;
}

/* ✅ RTL + Center ONLY for Arabic output */
.center-output.rtl textarea {
  direction: rtl !important;
  text-align: center !important;
}
"""




with gr.Blocks(css=css) as demo:
    main_container = gr.Column(elem_classes=[])

    with main_container:
        title_md = gr.Markdown(LANG["English"]["title"])
        desc_md = gr.Markdown(LANG["English"]["desc"])

        language = gr.Radio(
            ["English", "العربية"],
            label="Language / اللغة",
            value="English"
        )

        gender = gr.Radio(
            LANG["English"]["gender_choices"],
            label=LANG["English"]["gender"]
        )

        age = gr.Number(label=LANG["English"]["age"])
        height = gr.Number(label=LANG["English"]["height"])
        weight = gr.Number(label=LANG["English"]["weight"])

        goal = gr.Dropdown(
            choices=LANG["English"]["goal_choices"],
            label=LANG["English"]["goal"]
        )

        days = gr.Slider(3, 6, step=1, value=3, label=LANG["English"]["days"])

        experience = gr.Dropdown(
            choices=LANG["English"]["experience_choices"],
            label=LANG["English"]["experience"]
        )

        diet = gr.Dropdown(
            choices=LANG["English"]["diet_choices"],
            label=LANG["English"]["diet"]
        )

        submit_btn = gr.Button(LANG["English"]["submit"])
        clear_btn = gr.Button(LANG["English"]["clear"])

        output = gr.Textbox(
           label=LANG["English"]["output"],
           lines=30,
           elem_classes=["center-output"])


        submit_btn.click(
            fn=app_ui,
            inputs=[language, gender, age, height, weight, goal, days, experience, diet],
            outputs=output
        )

        clear_btn.click(lambda: "", outputs=output)


        def switch_language(lang):
            rtl = ["rtl"] if lang == "العربية" else []

            return (
               LANG[lang]["title"],
               LANG[lang]["desc"],

               gr.update(label=LANG[lang]["gender"],
                         choices=LANG[lang]["gender_choices"]),

               gr.update(label=LANG[lang]["age"]),
               gr.update(label=LANG[lang]["height"]),
               gr.update(label=LANG[lang]["weight"]),

               gr.update(label=LANG[lang]["goal"],
                         choices=LANG[lang]["goal_choices"]),

               gr.update(label=LANG[lang]["days"]),

               gr.update(label=LANG[lang]["experience"],
                        choices=LANG[lang]["experience_choices"]),

               gr.update(label=LANG[lang]["diet"],
                         choices=LANG[lang]["diet_choices"]),

               gr.update(value=LANG[lang]["submit"]),
               gr.update(value=LANG[lang]["clear"]),
               gr.update(label=LANG[lang]["output"]),

               # ✅ APPLY RTL TO FORM CONTAINER ONLY
               gr.update(elem_classes=rtl),
    )




    language.change(
    fn=switch_language,
    inputs=language,
    outputs=[
        title_md,
        desc_md,
        gender,
        age,
        height,
        weight,
        goal,
        days,
        experience,
        diet,
        submit_btn,
        clear_btn,
        output,
        main_container,   # ✅ THIS IS WHAT ENABLES RTL
    ]
)


demo.launch(share=True)


/tmp/ipython-input-2755597526.py:339: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30a7797b78c91c1c31.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
